In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os

sys.path.append("../src")  # adjust path as needed
from read_Parkinsonpredict import ReadData, LoadData
from ml_models import MlModels

In [24]:
directory_path = '../../../Data_sets/Quick_Start/complete_data/'
path = '../../../Data_sets/Quick_Start/Data/'

path_list = ["gene_expression_summary.csv","MDS_UPDRS_III_filtered_with_event.csv",
             "PDMRI_Clean_Merged_6_13_25.csv"]

for i in range(len(path_list)):
            path_list[i] = directory_path + path_list[i]

df_list = []

for s in path_list:
    i = path_list.index(s)
    D = pd.read_csv(s)
    df_list.append(D)

#print(df_list[1])

df_UPDRS_III = pd.read_csv(path + "MDS-UPDRS_Part_III_12Jun2025.csv")
df_UPDRS_IV = pd.read_csv(path + "MDS-UPDRS_Part_IV__Motor_Complications_12Jun2025.csv")
df_UPDRS_I = pd.read_csv(path + "MDS-UPDRS_Part_I_12Jun2025.csv")
print(df_UPDRS_III.columns)  

/var/folders/zb/s9y25xv57sd631cqzphbhf4r0000gn/T/ipykernel_69332/511909534.py:19: DtypeWarning: Columns (16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_UPDRS_III = pd.read_csv(path + "MDS-UPDRS_Part_III_12Jun2025.csv")


Index(['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'PDTRTMNT',
       'PDSTATE', 'HRPOSTMED', 'HRDBSON', 'HRDBSOFF', 'PDMEDYN', 'DBSYN',
       'ONOFFORDER', 'OFFEXAM', 'OFFNORSN', 'DBSOFFYN', 'DBSOFFTM', 'ONEXAM',
       'ONNORSN', 'HIFUYN', 'DBSONYN', 'DBSONTM', 'PDMEDDT', 'PDMEDTM',
       'EXAMDT', 'EXAMTM', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU',
       'NP3RIGLU', 'NP3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR',
       'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR',
       'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR',
       'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU',
       'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'NP3TOT',
       'DYSKPRES', 'DYSKIRAT', 'NHY', 'ORIG_ENTRY', 'LAST_UPDATE'],
      dtype='object')


In [7]:
from functools import reduce

merged_df = reduce(
    lambda left, right: pd.merge(
        left, right, on=['PATNO', 'EVENT_ID'], how='inner', suffixes=('', '_drop')
    ),
    df_list
)

merged_df = merged_df.drop(columns=['NHY_drop'])

In [8]:
print(merged_df.EVENT_ID.unique())
print(merged_df.info())
print(merged_df.describe(include='all'))

['BL']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PATNO                          374 non-null    int64  
 1   EVENT_ID                       374 non-null    object 
 2   ENSG00000188906                374 non-null    float64
 3   ENSG00000185345                374 non-null    float64
 4   ENSG00000158828                374 non-null    float64
 5   ENSG00000145335                374 non-null    float64
 6   ENSG00000159082                374 non-null    float64
 7   ENSG00000116288                374 non-null    float64
 8   ENSG00000116675                374 non-null    float64
 9   ENSG00000177628                374 non-null    float64
 10  ENSG00000100225                374 non-null    float64
 11  ENSG00000184381                374 non-null    float64
 12  NHY                            374 non-null

In [10]:
loaddata = LoadData(input_path=directory_path)
X_train, Y_train = loaddata.merged_data()

Is PATNO sorted?: True
Do we have all unique patients?: Yes
The train and test data is split according to 80.0 - 20.0%


In [17]:
print(X_train.columns)
print(Y_train.columns)

Index(['lh_MeanThickness', 'rh_MeanThickness', 'lh_WhiteSurfArea',
       'rh_WhiteSurfArea', 'rhCerebralWhiteMatterVol',
       'lhCerebralWhiteMatterVol', 'Left_Cerebellum_White_Matter',
       'Right_Cerebellum_White_Matter', 'CerebralWhiteMatterVol',
       'rhSurfaceHoles', 'lhSurfaceHoles', 'ENSG00000188906',
       'ENSG00000185345', 'ENSG00000158828', 'ENSG00000145335',
       'ENSG00000159082', 'ENSG00000116288', 'ENSG00000116675',
       'ENSG00000177628', 'ENSG00000100225', 'ENSG00000184381'],
      dtype='object')
Index(['NHY'], dtype='object')


## Practice cells below

### Commented cell

1. MRI score vs NHY

D = merged_df.MRIRSLT-merged_df.NHY
plt.figure(figsize=(8, 5))
plt.hist(D, bins=20, color='skyblue', edgecolor='black')
plt.xlabel('NHY - MRIRSLT')
plt.ylabel('Frequency')
plt.title('Histogram of NHY and MRIRSLT Differences')
plt.tight_layout()
plt.show()

1. MRI measurements vs NHY

cols_to_plot = [col for col in merged_df.columns if col.lower().endswith('_meanthickness') or 
                col.lower().endswith('_whitesurfarea')]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, col in enumerate(cols_to_plot):
    scatter = axes[idx].scatter(
        merged_df[col], merged_df['MRIRSLT'],
        c=merged_df['NHY'], cmap='viridis', alpha=0.7, edgecolor='k'
    )
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('MRIRSLT')
    axes[idx].set_title(f'{col} vs MRIRSLT')
    cbar = fig.colorbar(scatter, ax=axes[idx])
    cbar.set_label('NHY')

plt.tight_layout()
plt.show()

In [72]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
Feature_columns = ["PATNO", "MRIRSLT" ]
#Feature_columns = ["PATNO", "ENSG00000188906", "ENSG00000185345", "ENSG00000158828", "ENSG00000145335", "ENSG00000159082","ENSG00000116288", "ENSG00000116675", "ENSG00000177628","ENSG00000100225", "ENSG00000184381","MRIRSLT" ]
# "lh_MeanThickness","rh_MeanThickness", "lh_WhiteSurfArea","rh_WhiteSurfArea", "rhCerebralWhiteMatterVol"

In [ ]:
X = merged_df[Feature_columns]
y = merged_df['NHY']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=142, stratify=y
)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


y_pred = knn.predict(X_test)


print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.54      0.52        28
         1.0       0.50      0.45      0.47        20
         2.0       0.44      0.44      0.44        27

    accuracy                           0.48        75
   macro avg       0.48      0.48      0.48        75
weighted avg       0.48      0.48      0.48        75

Accuracy: 0.48
